In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression

## Load data

In [2]:
df_train = pd.read_csv("data/ADNIMERGE_train.csv")
df_test = pd.read_csv("data/ADNIMERGE_test.csv")

In [3]:
X_train = df_train.drop(['RID', 'DX_bl'], axis=1).copy()
y_train = df_train['DX_bl'].copy()
X_test = df_test.drop(['RID', 'DX_bl'], axis=1).copy()
y_test = df_test['DX_bl'].copy()

## Significance of Predictors

We would like to find out the most significant variables in the model. There variables have the strongest predicting power, and are thus the most useful in the diagnosis of Alzheimer's disease. Identifying these variables can eliminate the number of tests a patient has to go through to accurately diagnose AD. 

For logistic regression with l1 regularization, we used bootstraping (1000 iterations) to find the most significant predictors. For random forest, we used the returned attribute `feature_importances_`.

### Bootstrap

In [4]:
log_l1 = LogisticRegressionCV(penalty = 'l1', solver = 'liblinear')
log_l1.fit(X_train,y_train)
c = log_l1.C_[0]


iterations = 200
boot = np.zeros((X_train.shape[1], iterations))
for i in range(iterations):
    boot_rows = np.random.choice(range(X_train.shape[0]),
                                 size=X_train.shape[0], replace=True)
    X_train_boot = X_train.values[boot_rows]
    y_train_boot = y_train.values[boot_rows]
    model_boot = LogisticRegression(penalty = 'l1', C=c)
    model_boot.fit(X_train_boot, y_train_boot)
    boot[:,i] = model_boot.coef_[2,:]
    
boot_ci_upper = np.percentile(boot, 97.5, axis=1)
boot_ci_lower = np.percentile(boot, 2.5, axis=1)
sig_b_ct = []
for i in range(X_train.shape[1]):
    if boot_ci_upper[i]<0 or boot_ci_lower[i]>0:
        sig_b_ct.append(i)
        
print("Most significant coefficients: ")
print(X_train.columns[sig_b_ct])

Most significant coefficients: 
Index(['ADAS13', 'MMSE', 'FAQ'], dtype='object')


Surprisingly, we only have 3 significant predictors using bootstrap method, `ADAS13` and `MMSE` (Mini-Mental State Examination) are both cognitive assessments which are likely to be free or cheap. 

### Feature importance

In [5]:
rf_best = RandomForestClassifier(n_estimators=32, max_depth=6)
rf_best.fit(X_train, y_train)
imp_features = np.array(X_train.columns)[rf_best.feature_importances_!=0]
print("The most important {} features:".format(len(imp_features)))
print(imp_features)

The most important 70 features:
['PTGENDER' 'PTEDUCAT' 'PTRACCAT_Asian' 'PTRACCAT_Black' 'PTRACCAT_White'
 'PTETHCAT_Not_Hisp/Latino' 'PTMARRY_Never_married' 'PTMARRY_Widowed'
 'APOE4' 'CSF_ABETA' 'CSF_TAU' 'CSF_PTAU' 'FDG' 'FDG_slope' 'AV45'
 'AV45_slope' 'ADAS13' 'ADAS13_slope' 'MMSE' 'MMSE_slope' 'RAVLT_immediate'
 'RAVLT_immediate_slope' 'RAVLT_learning' 'RAVLT_learning_slope'
 'RAVLT_forgetting' 'RAVLT_forgetting_slope' 'RAVLT_perc_forgetting'
 'RAVLT_perc_forgetting_slope' 'MOCA' 'MOCA_slope' 'EcogPtMem'
 'EcogPtMem_slope' 'EcogPtLang' 'EcogPtLang_slope' 'EcogPtVisspat'
 'EcogPtVisspat_slope' 'EcogPtPlan' 'EcogPtPlan_slope' 'EcogPtOrgan'
 'EcogPtOrgan_slope' 'EcogPtDivatt' 'EcogPtDivatt_slope' 'EcogSPMem'
 'EcogSPMem_slope' 'EcogSPLang' 'EcogSPLang_slope' 'EcogSPVisspat'
 'EcogSPVisspat_slope' 'EcogSPPlan' 'EcogSPPlan_slope' 'EcogSPOrgan'
 'EcogSPOrgan_slope' 'EcogSPDivatt' 'EcogSPDivatt_slope' 'FAQ' 'FAQ_slope'
 'Ventricles' 'Ventricles_slope' 'Hippocampus' 'Hippocampus_slope'
 

Using random forest classifier, we ended up with 70 important features. If the slope of certain variable is not an important feature, we can at least avoid going through the same test again and again in each visit.

## Forward and Backward Selection

In [6]:
def step_forwards_backwards(direction='forward'):
    
    assert direction in ['forward', 'backward']

    predictors = set(X_train.columns)
    selected_predictors = set() if direction=='forward' else set(predictors)
    
    n = X_train.shape[0]
    best_acc = np.inf
    
    best_accuracy = []
    best_models = []
    
    if direction == 'forward':
        X = X_train[list(selected_predictors)].values
        while (True):
            
            possible_scores = []
            possible_predictors = list(selected_predictors ^ predictors)
            
            if len(possible_predictors) == 0:
                break
                
            for predictor in possible_predictors:
                x_temp = np.concatenate([X, X_train[predictor].values.reshape(-1,1)], axis=1)
                rf = RandomForestClassifier(n_estimators=32, max_depth=6)
                rf.fit(x_temp, y_train)
                scores = rf.score(x_temp, y_train)
                possible_scores.append(scores)
                
            best_predictor_ix = np.argmax(possible_scores)
            best_predictor = possible_predictors[best_predictor_ix]
            
            best_acc = np.max(possible_scores)
            best_accuracy.append(best_acc)
            
            selected_predictors.add(best_predictor)            
            X = np.concatenate([X, X_train[best_predictor].values.reshape(-1,1)], axis=1)
            best_models.append(list(selected_predictors))

    else:

        while (True):
            possible_scores = []
            possible_predictors = list(selected_predictors)

            if len(possible_predictors) == 0:
                break

            for predictor in possible_predictors:
                X = np.concatenate([np.ones(n).reshape(-1,1), 
                                    X_train[list(selected_predictors - set([predictor]))].values], 
                                   axis=1)
                if(X.shape[1] != 0):
                    rf = RandomForestClassifier(n_estimators=32, max_depth=6)
                    rf.fit(X, y_train)
                    scores = rf.score(X, y_train)
                    possible_scores.append(scores)

            best_predictor_ix = np.argmax(possible_scores)
            best_predictor = possible_predictors[best_predictor_ix] 

            best_acc = possible_scores[best_predictor_ix]
            selected_predictors.discard(best_predictor)
            
            best_accuracy.append(best_acc)
            best_models.append(list(selected_predictors))
            
    index_of_best_accuracy = np.argmax(best_accuracy)

    return best_models[index_of_best_accuracy]

In [7]:
predictors_forward = step_forwards_backwards(direction='forward')
predictors_backward = step_forwards_backwards(direction='backward')
print("Predictors selected by forward selection (", 
      len(predictors_forward), " predictors): \n", predictors_forward)
print("\n-----------------------------------------\n")
print("Predictors selected by backward selection: (", 
      len(predictors_forward), " predictors): \n", predictors_backward)

Predictors selected by forward selection ( 29  predictors): 
 ['APOE4', 'PTMARRY_Married', 'CSF_ABETA', 'MOCA_slope', 'EcogPtLang', 'Ventricles', 'MMSE', 'MidTemp', 'WholeBrain', 'WholeBrain_slope', 'Entorhinal_slope', 'EcogSPLang', 'FAQ', 'PTEDUCAT', 'EcogPtOrgan_slope', 'FAQ_slope', 'EcogPtMem', 'Ventricles_slope', 'EcogPtLang_slope', 'Entorhinal', 'RAVLT_forgetting_slope', 'EcogPtVisspat_slope', 'FDG', 'FDG_slope', 'CSF_PTAU', 'Fusiform_slope', 'EcogPtDivatt', 'EcogSPMem', 'PTRACCAT_Black']

-----------------------------------------

Predictors selected by backward selection: ( 29  predictors): 
 ['PTMARRY_Married', 'EcogSPMem_slope', 'PTRACCAT_Unknown', 'CSF_ABETA', 'MOCA_slope', 'EcogPtPlan', 'PTRACCAT_Asian', 'RAVLT_learning_slope', 'MMSE', 'RAVLT_immediate', 'EcogSPPlan_slope', 'PTRACCAT_White', 'MOCA', 'WholeBrain_slope', 'EcogPtDivatt_slope', 'ADAS13', 'Hippocampus', 'EcogPtMem', 'RAVLT_forgetting', 'EcogPtLang_slope', 'Entorhinal', 'RAVLT_forgetting_slope', 'EcogSPVisspat_slo

We can see that genetic analysis such as `APOE4`, CSF biosamples, neuropsychological tests and MRI are the most important varibles in predicting AD. However, only a few variables we get from each of tests are useful, so we do not need to focus on all the testing results. 

Also, notably, we found that it is necessary to take the cognitive assessments and perform brain scan multiple times to check the progress of cognitive decline and brain atrophy. These are very indicative of AD. For the other categories, one test is sufficient for the diagnosis of AD. Going through the same test multiple times will not increase predictability.